In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [45]:
class SklearnLGBM(LGBMClassifier):
    def __sklearn_tags__(self):
        return {"estimator_type": "classifier", "requires_y": True}

class SklearnXGB(XGBClassifier):
    def __sklearn_tags__(self):
        return {"estimator_type": "classifier", "requires_y": True}


In [46]:
df = pd.read_csv("cleaned_data.csv")

X = df.drop(columns=['class'])
y = df['class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [47]:
model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lgb = SklearnLGBM(random_state=42)
model_xgb = SklearnXGB(use_label_encoder=False, eval_metric='mlogloss', random_state=42)


In [48]:
voting_clf = VotingClassifier(
    estimators=[
        ('lr', model_lr),
        ('lgb', model_lgb),
        ('xgb', model_xgb)
    ],
    voting='soft'  # soft da artık kullanılabilir
)


In [50]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_gb = GradientBoostingClassifier(n_estimators=100, random_state=42)

voting_clf = VotingClassifier(
    estimators=[
        ('lr', model_lr),
        ('rf', model_rf),
        ('gb', model_gb)
    ],
    voting='soft'
)


In [51]:
voting_clf.fit(X_train, y_train)
y_pred = voting_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"VotingClassifier Accuracy: {accuracy:.4f}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier Accuracy: 0.7223
[[481  83  42   8]
 [ 46 425 127  58]
 [ 18  94 396 150]
 [  1   5  87 568]]
              precision    recall  f1-score   support

           1       0.88      0.78      0.83       614
           2       0.70      0.65      0.67       656
           3       0.61      0.60      0.60       658
           4       0.72      0.86      0.79       661

    accuracy                           0.72      2589
   macro avg       0.73      0.72      0.72      2589
weighted avg       0.73      0.72      0.72      2589

